## Problem Setup

Let's say we have a neural network like the one below:

<div style="background: #FFF;">
    <img width="100%" src="./nn.svg">
</div>

This network has three layers: an input layer, a hidden layer and an output layer. The size of each layer is 16, 12, and 10 respectively. This means that the network takes in an input with 16 features, compresses it to 12, and does a further compression to 10 (which is the output).

How many weights will this network have? To figure this out, we merely need to count the number of connections between all nodes. Between the input layer and the hidden layer, we will have $16 \times 12$ = 192 weights, while between the hidden and output layer, we will have $12 \times 10$ = 120 weights.

To keep our notations consistent, we'll define our matrices with rows being the number of inputs and columns being the number of outputs. If I had n samples, then I could create the following matrices:

- Input Data ($X$): $n \times 16$ matrix, where n is the number of samples you input in the network. Each sample should have 16 features in this example.
- 1st Weights ($W_1$): $16 \times 12$ matrix
- 2nd Weights ($W_2$): $12 \times 10$ matrix

Passing in the input data into our network, it should be easy to see that we should end up with an output matrix of n x 10 (do a quick check of dimensions for the matrix multiplication of $X W_1 W_2$).

## Training the network
  
On each iteration of your neural network you should be doing 3 things:

1. Do a feedforward pass. Using the current weights of the network, calculate predictions/outputs using your training data. You will compare these outputs to the ground truth labels in order to get an error that you will use for the 2nd step of training: backpropagation.

2. Backpropagate the error through the network. This will allow you to get the gradients for each sets of weights, so that you can correct the weights as a proportion of it's error in the output (this is essentially gradient descent).

3. With the updated weights, calculate predictions/outputs on your test set. Compare with the ground truth to get an overall error for the test set. Since these our examples our network has not seen before, it is desirable to check this error for a true assessment of how our network is performing. You can stop training when there is no more improvement in the testing error.

During training, it is more computationally efficient to vectorize each of these operations since training can take a while.

## On calculating the feedforwards outputs

Recall what a single neuron is doing. Let $\mathbf{w}$ be a vector of weights of the neuron, $\mathbf{x}$ be an input vector, and $\phi$ be an activation function (e.g. logistic function). Then we have:

$$
    y(x) = \phi(w^{T}x)
$$

However, we want to vectorize this operation across multiple examples and neurons. Referring to the matrices stated in the problem setup above this will lead to the following matrix calculation:

$$
    Y(X) = \phi(\phi(XW_1)W_2)
$$

As stated before, this will give you an output matrix of $n \times 10$, which you can use to calculate the error with respect to the ground truth training labels.

## On backpropagation

Backpropogation leads to a recursive gradient expression that such that the gradient for each layer relies on the subsequent layers closest to the output layer. 

To make this more concrete, let's use the logistic activation function as for our example. If this is the case, then we get the following gradient expressions:

- Gradients for output layer:

$$
    \delta_{ij} = (d_{i} - y_{ij}) y_{ij} (1 - y_{ij})
$$

where $y_{ij}$ refers to output of the $i$th neuron of the $j$th layer (but in this case $j$ is the last (output) layer of the network, and $d_i$ are the ground truth labels for the example. In matrix terms, this will give you:

$$
    \delta_j = (D - Y_j) \circ Y_j \circ (1 - Y_j)
$$

where $\circ$ is the Hadamard product (aka. element-wise product). For our example, it's understood that $D$ is a $n \times 10$ matrix of ground truths, while $Y_j$ is an $n \times 10$ output matrix from the network.

- Gradients for NOT output layer:

$$
    \delta_{ij} = y_{ij} (1 - y_{ij}) \sum_{i} \delta_{i(j+1)} w_{i(j+1)}
$$

This expression tells us that we take the gradient for each neuron the forward layer, and do an inner product with that neuron's weight vector. This value is further multiplied by the activation function gradient. In matrix terms, this can be expressed as:

$$
    \delta_j = Y_j \circ (1 - Y_j) \circ \delta_{j+1} W_{j+1}^{T}
$$

## Getting the full gradient

To get the full gradient for each sets of weights, we need to multiply the $\delta$ terms by the output of the previous layer before it:

- Gradient for layer where input is data (j=0):

$$
    \frac{\partial E}{W_j} = X^{T} \delta_j 
$$

- Gradient for layer where input is another layer (j>0):

$$
    \frac{\partial E}{W_j} = Y_{j-1}^{T} \delta_j 
$$

For this particular example, I have two sets of weights which gives me two sets of gradients. By following the above, the dims of $\delta_j$ for the hidden and output layers are $n \times 12$ and $n \times 10$ respectively. This leads to gradient matrices for $W_1$ and $W_2$ to be $16 \times 12 = (16 \times n)(n \times 12)$ and $12 \times 10 = (12 \times n)(n \times 10)$ respectively. In other words, if you did your calculations correctly, the gradient expression should collapse along the number of samples dimension. This means that your gradient expression should be valid for any choice of $n$!

Once the gradients are found, we should update the weights using a gradient descent step:

$$
    W_j(n) = W_j(n-1) + \eta \frac{\partial E}{W_j}
$$

where $n$ denotes the iteration of the update.